In [13]:
import pygame
import os
import time
import random
from tkinter import *
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
from mutagen.mp3 import MP3
import webbrowser
import json


# setting

In [ ]:
pygame.init()
pygame.mixer.init()

# متغیرهای سراسری برای کنترل پنجره‌ها
music_window = None
alarm_window = None

def music():
    global music_window
    if music_window is not None and music_window.winfo_exists():
        music_window.lift()
        return  # اگر باز است، فقط می‌آوردش جلو
    
    music_window = Toplevel()
    music_window.title("🎵 موزیک پلیر حرفه‌ای 🎵")
    music_window.geometry("575x575")
    music_window.configure(bg="#222")
    music_window.attributes("-topmost", True)

    class MusicPlayer:
        def __init__(self, root):
            self.root = root
            self.playlist = []
            self.current_index = 0
            self.paused = False
            self.song_length = 0
            self.start_time = 0
            self.offset = 0
            self.shuffle = False
            self.repeat = "None"
            self.playlist_file = os.path.join(os.getcwd(), "playlist.json")
            self.load_playlist_file()

            self.label = Label(root, text="هیچ آهنگی انتخاب نشده", bg="#222", fg="white", font=("Arial", 14))
            self.label.pack(pady=10)

            list_frame = Frame(root, bg="#222")
            list_frame.pack(pady=5)
            self.listbox = Listbox(list_frame, bg="#333", fg="white", width=60, height=12,
                                   selectbackground="#555", font=("Arial", 12))
            self.listbox.pack(side=LEFT, fill=BOTH, expand=True)
            self.listbox.bind('<Double-Button-1>', self.play_selected)
            self.scrollbar = Scrollbar(list_frame)
            self.scrollbar.pack(side=RIGHT, fill=Y)
            self.listbox.config(yscrollcommand=self.scrollbar.set)
            self.scrollbar.config(command=self.listbox.yview)

            for song in self.playlist:
                self.listbox.insert(END, os.path.basename(song))

            control_frame = Frame(root, bg="#222")
            control_frame.pack(pady=5)
            Button(control_frame, text="📂 افزودن آهنگ", command=self.add_songs,
                   bg="#29E92F", fg="white", font=("Arial", 12)).grid(row=0, column=0, padx=5)
            Button(control_frame, text="🗑 حذف آهنگ", command=self.remove_song,
                   bg="#f44336", fg="white", font=("Arial", 12)).grid(row=0, column=1, padx=5)
            Button(control_frame, text="◀ قبلی", command=self.prev_song,
                   bg="#2196F3", fg="white", font=("Arial", 12)).grid(row=0, column=2, padx=5)
            Button(control_frame, text="▶ بعدی", command=self.next_song,
                   bg="#2196F3", fg="white", font=("Arial", 12)).grid(row=0, column=3, padx=5)
            Button(control_frame, text="⏸ توقف/ادامه", command=self.pause_resume,
                   bg="#f44336", fg="white", font=("Arial", 12)).grid(row=0, column=4, padx=5)
            Button(control_frame, text="⏹ توقف", command=self.stop_song,
                   bg="#9C27B0", fg="white", font=("Arial", 12)).grid(row=0, column=5, padx=5)

            self.volume_scale = Scale(root, from_=0, to=100, orient="horizontal", label="🔊 صدا",
                                      command=self.set_volume, bg="#222", fg="white", length=200, width=10)
            self.volume_scale.set(70)
            self.volume_scale.pack(pady=5)
            pygame.mixer.music.set_volume(0.7)

            self.progress = Scale(root, from_=0, to=100, orient="horizontal", length=500,
                                  bg='#222', fg="white", width=10, label="نوار پیشرفت")
            self.progress.pack(pady=5)
            self.progress.bind("<ButtonRelease-1>", self.seek_release)

            self.time_label = Label(root, text="00:00 / 00:00", bg="#222", fg="white", font=("Arial", 12))
            self.time_label.pack(pady=5)

            options_frame = Frame(root, bg="#222")
            options_frame.pack(pady=5)
            pooya = Frame(root, bg="#222")
            pooya.pack(pady=5)
            self.shuffle_btn = Button(options_frame, text="Shuffle Off", command=self.toggle_shuffle, bg="#555", fg="white")
            self.shuffle_btn.grid(row=0, column=0, padx=5)
            self.repeat_btn = Button(options_frame, text="Repeat: None", command=self.toggle_repeat, bg="#555", fg="white")
            self.repeat_btn.grid(row=0, column=1, padx=5)
            self.name_btn = Button(pooya, text="      𝓶𝓪𝓭𝓮 𝓫𝔂 𝓟𝓸𝓸𝔂𝓪      ",
                                   command=self.open_github, font=(12), bg="#222", fg="white")
            self.name_btn.grid(row=0, column=0, padx=5)

            self.update_progress()
            self.root.protocol("WM_DELETE_WINDOW", self.on_close)

        def add_songs(self):
            files = filedialog.askopenfilenames(title="انتخاب آهنگ", filetypes=[("Audio Files", "*.mp3 *.wav")])
            if files:
                for f in files:
                    if f not in self.playlist:
                        self.playlist.append(f)
                        self.listbox.insert(END, os.path.basename(f))
                self.save_playlist_file()
                if len(files) > 0 and not pygame.mixer.music.get_busy():
                    self.current_index = 0
                    self.load_song(self.playlist[self.current_index])

        def remove_song(self):
            selection = self.listbox.curselection()
            if selection:
                index = selection[0]
                del self.playlist[index]
                self.listbox.delete(index)
                self.save_playlist_file()
                if index == self.current_index:
                    self.stop_song()

        def save_playlist_file(self):
            try:
                with open(self.playlist_file, "w", encoding="utf-8") as f:
                    json.dump(self.playlist, f, ensure_ascii=False, indent=4)
            except Exception as e:
                messagebox.showerror("خطا", f"ذخیره لیست پخش امکان‌پذیر نبود:\n{e}")

        def load_playlist_file(self):
            if os.path.exists(self.playlist_file):
                try:
                    with open(self.playlist_file, "r", encoding="utf-8") as f:
                        self.playlist = json.load(f)
                except Exception as e:
                    messagebox.showwarning("خطا", f"بارگذاری لیست پخش موفق نبود:\n{e}")
                    self.playlist = []

        def load_song(self, file):
            self.current_file = file
            self.label.config(text=f"در حال پخش: {os.path.basename(file)}")
            try:
                if file.lower().endswith(".mp3"):
                    audio = MP3(file)
                    self.song_length = int(audio.info.length)
                else:
                    snd = pygame.mixer.Sound(file)
                    self.song_length = int(snd.get_length())
                self.progress.config(to=self.song_length)
                self.offset = 0
                pygame.mixer.music.load(file)
                pygame.mixer.music.play()
                self.start_time = time.time()
                self.paused = False

                self.listbox.selection_clear(0, END)
                self.listbox.selection_set(self.current_index)
                self.listbox.activate(self.current_index)
            except Exception as e:
                messagebox.showerror("خطا", f"مشکل در پخش: {e}")

        def play_selected(self, event):
            selection = self.listbox.curselection()
            if selection:
                self.current_index = selection[0]
                self.load_song(self.playlist[self.current_index])

        def pause_resume(self):
            if not self.playlist: return
            if not self.paused:
                pygame.mixer.music.pause()
                self.paused = True
                self.offset = time.time() - self.start_time
            else:
                pygame.mixer.music.unpause()
                self.paused = False
                self.start_time = time.time() - self.offset

        def stop_song(self):
            if not self.playlist: return
            pygame.mixer.music.stop()
            self.paused = False
            self.offset = 0
            self.progress.set(0)

        def next_song(self):
            if not self.playlist: return
            if self.shuffle:
                self.current_index = random.randint(0, len(self.playlist) - 1)
            else:
                self.current_index += 1
                if self.current_index >= len(self.playlist):
                    if self.repeat == "All":
                        self.current_index = 0
                    else:
                        self.current_index = len(self.playlist) - 1
            self.load_song(self.playlist[self.current_index])

        def prev_song(self):
            if not self.playlist: return
            if self.shuffle:
                self.current_index = random.randint(0, len(self.playlist) - 1)
            else:
                self.current_index -= 1
                if self.current_index < 0:
                    if self.repeat == "All":
                        self.current_index = len(self.playlist) - 1
                    else:
                        self.current_index = 0
            self.load_song(self.playlist[self.current_index])

        def set_volume(self, val):
            pygame.mixer.music.set_volume(int(val) / 100)

        def update_progress(self):
            if self.playlist and (pygame.mixer.music.get_busy() or self.paused):
                if self.paused:
                    pos = int(self.offset)
                else:
                    pos = int(time.time() - self.start_time)
                    if pos > self.song_length:
                        if self.repeat == "One":
                            self.load_song(self.current_file)
                            pos = 0
                        else:
                            if self.shuffle:
                                self.current_index = random.randint(0, len(self.playlist) - 1)
                            else:
                                self.current_index += 1
                                if self.current_index >= len(self.playlist):
                                    if self.repeat == "All":
                                        self.current_index = 0
                                    else:
                                        self.current_index = len(self.playlist) - 1
                            self.load_song(self.playlist[self.current_index])
                            pos = 0

                self.progress.set(pos)
                elapsed = time.strftime("%M:%S", time.gmtime(pos))
                total = time.strftime("%M:%S", time.gmtime(self.song_length))
                self.time_label.config(text=f"{elapsed} / {total}")
            self.root.after(500, self.update_progress)

        def seek_release(self, event):
            if self.playlist:
                pos = int(self.progress.get())
                self.offset = pos
                pygame.mixer.music.stop()
                pygame.mixer.music.play(start=self.offset)
                self.start_time = time.time() - self.offset
                self.paused = False

        def toggle_shuffle(self):
            self.shuffle = not self.shuffle
            self.shuffle_btn.config(text=f"Shuffle {'On' if self.shuffle else 'Off'}")

        def toggle_repeat(self):
            if self.repeat == "None":
                self.repeat = "One"
            elif self.repeat == "One":
                self.repeat = "All"
            else:
                self.repeat = "None"
            self.repeat_btn.config(text=f"Repeat: {self.repeat}")

        def on_close(self):
            pygame.mixer.music.stop()
            global music_window
            music_window = None
            self.root.destroy()

        def open_github(self):
            url = "https://github.com/PooyaLazuli"
            webbrowser.open_new_tab(url)

    MusicPlayer(music_window)

def alarm_setter():
    global alarm_window
    if alarm_window is not None and alarm_window.winfo_exists():
        alarm_window.lift()
        return
    
    alarm_window = Toplevel()
    alarm_window.config(bg="#222")
    alarm_window.title("Alarm Settings")
    
    def save_alarm():
        global hour1, minute1, second1
        h = entry_hour.get()
        m = entry_min.get()
        s = entry_sec.get()

        h = h.zfill(2)
        m = m.zfill(2)
        s = s.zfill(2)

        hour1, minute1, second1 = str(h), str(m), str(s)
        lbl_set.config(text=f"⏰ زمان ثبت شد: {hour1}:{minute1}:{second1}", bg="#222", fg="white")

    frame = Frame(alarm_window, bg="#222")
    frame.pack(pady=10)

    Label(frame, text="ساعت:", bg="#222", fg="white").grid(row=0, column=0)
    entry_hour = Entry(frame, width=5)
    entry_hour.grid(row=0, column=1)

    Label(frame, text="دقیقه:", bg="#222", fg="white").grid(row=0, column=2)
    entry_min = Entry(frame, width=5)
    entry_min.grid(row=0, column=3)

    Label(frame, text="ثانیه:", bg="#222", fg="white").grid(row=0, column=4)
    entry_sec = Entry(frame, width=5)
    entry_sec.grid(row=0, column=5)

    btn_set = Button(frame, text="ثبت زمان", bg="#222", fg="white", command=save_alarm)
    btn_set.grid(row=0, column=6, padx=10)

    lbl_set = Label(alarm_window, text="⏳ هنوز زمانی ثبت نشده", bg="#222", fg="white")
    lbl_set.pack(pady=10)

    def on_close_alarm():
        global alarm_window
        alarm_window.destroy()
        alarm_window = None

    alarm_window.protocol("WM_DELETE_WINDOW", on_close_alarm)

: 

# loop

In [ ]:


loop = Tk()
loop.configure(bg="#222")
loop.title('Clock Practice')
loop.geometry('200x100')
loop.attributes("-topmost", True)

num = 0
hour1 = '00'
minute1 = '00'
second1 = '00'

# دکمه‌ها
Button(loop, text="music", command=music).place(rely=0.85, relx=0.25, anchor="center")
Button(loop, text="setting", command=alarm_setter).place(rely=0.85, relx=0.75, anchor="center")

day = time.strftime("%A")
zone = time.strftime("%Z")
mytext2 = day + "   " + zone

lable1 = Label(loop, text='testiiii', font=('Arial', 24), fg='green', bg="#222")
lable1.pack()
lable2 = Label(loop, text=mytext2, font=('Arial', 12), fg='white', bg="#222")
lable2.pack()

def mytime():
    global num, hour1, minute1, second1
    hour = time.strftime('%I')
    minute = time.strftime("%M")
    second = time.strftime("%S")
    am_pm = time.strftime("%p")

    if hour == hour1 and minute == minute1 and second == second1:
        lable2.config(text='⏰ زمان آلارم رسید!', fg="#FA0DFA")
    if hour != hour1 and minute != minute1:
        lable2.config(text=mytext2, font=('Arial', 12), fg='white', bg="#222",)

    mytext = f"{hour}:{minute}:{second} {am_pm}"
    rand_list = ['red', "#29E92F", "#FA0DFA", "#22DFEC", "#F6FA0D"]
    rand = rand_list[num]
    lable1.config(text=mytext, fg=rand)
    num = (num + 1) % len(rand_list)
    lable1.after(1000, mytime)

def on_main_close():
    global music_window, alarm_window
    if music_window and music_window.winfo_exists():
        music_window.destroy()
    if alarm_window and alarm_window.winfo_exists():
        alarm_window.destroy()
    loop.destroy()

loop.protocol("WM_DELETE_WINDOW", on_main_close)

mytime()
loop.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\Click\AppData\Local\Programs\Python\Python39\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\Click\AppData\Local\Temp\ipykernel_7208\588814823.py", line 50, in on_main_close
    loop.destroy()
  File "c:\Users\Click\AppData\Local\Programs\Python\Python39\lib\tkinter\__init__.py", line 2312, in destroy
    self.tk.call('destroy', self._w)
_tkinter.TclError: can't invoke "destroy" command: application has been destroyed
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\Click\AppData\Local\Programs\Python\Python39\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\Click\AppData\Local\Temp\ipykernel_7208\588814823.py", line 50, in on_main_close
    loop.destroy()
  File "c:\Users\Click\AppData\Local\Programs\Python\Python39\lib\tkinter\__init__.py", line 2312, in destroy
    self.tk.call('d

# time